# Библиотеки Python для Data Science: Numpy, Matplotlib, Scikit-learn
# Курсовой Проект Ершова Андрея Александровича

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 100

In [3]:
DATASET_PATH = 'c:\\Users\\andre\\source\\repos\\python\\DataScience\\project_task\\train.csv'
df = pd.read_csv(DATASET_PATH)

###### Описание датасета:

$Id$ : идентификационный номер квартиры
<br>
$DistrictId$ : идентификационный номер района
<br>
$Rooms$ : количество комнат
<br>
$Square$ : площадь
<br>
$LifeSquare$ : жилая площадь
<br>
$KitchenSquare$ : площадь кухни
<br>
$Floor$ : этаж
<br>
$HouseFloor$ : количество этажей в доме
<br>
$HouseYear$ : год постройки дома
<br>
$Ecology\_1, Ecology\_2, Ecology\_3$ : экологические показатели местности
<br>
$Social\_1, Social\_2, Social\_3$ : социальные показатели местности
<br>
$Healthcare\_1, Helthcare\_2$ : показатели местности, связанные с охраной здоровья
<br>
$Shops\_1, Shops\_2$ : показатели, связанные с наличием магазинов, торговых центров
<br>
$Price$ : цена квартиры

In [43]:
df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


Признаки $LifeSquare$ и $Healthcare\_1$ имеют высокую степень разряжения. Заполняем отсутсвующие в этих столбцах значения соотвествующими медианными значениями. 

In [92]:
#df.loc[df['LifeSquare'].isnull(), 'LifeSquare'] = df['LifeSquare'].median()
#df.loc[df['Healthcare_1'].isnull(), 'Healthcare_1'] = df['Healthcare_1'].median()
lsm = df['LifeSquare'].median()
df["LifeSquare"] = df["LifeSquare"].fillna(lsm)

hcm = df['Healthcare_1'].median()
df["Healthcare_1"] = df["Healthcare_1"].fillna(hcm)

Далее разберёмся с данными в колонках типа $object$.
<br>
Преобразуем категориальные признаки $Ecology\_2$, $Ecology\_3$ и $Shops\_2$ в несколько бинарных

In [129]:
matching = filter(lambda column: 'Ecology_2A' in column, df.columns)

if len(list(matching)) < 1:
    df = pd.concat([df, pd.get_dummies(df['Ecology_2'])], axis=1)
    df.rename(columns={"A": "Ecology_2A", "B": "Ecology_2B"}, inplace=True)
    df.drop("Ecology_2", axis=1, inplace=True)

In [130]:
matching = filter(lambda column: 'Ecology_3A' in column, df.columns)

if len(list(matching)) < 1:
    df = pd.concat([df, pd.get_dummies(df['Ecology_3'])], axis=1)
    df.rename(columns={"A": "Ecology_3A", "B": "Ecology_3B"}, inplace=True)
    df.drop("Ecology_3", axis=1, inplace=True)

In [131]:
matching = filter(lambda column: 'Shops_2A' in column, df.columns)

if len(list(matching)) < 1:
    df = pd.concat([df, pd.get_dummies(df['Shops_2'])], axis=1)
    df.rename(columns={"A": "Shops_2A", "B": "Shops_2B"}, inplace=True)
    df.drop("Shops_2", axis=1, inplace=True)

Избавляемся от $мультиколлинеарности$.
<br>


In [142]:
matching = filter(lambda column: 'Shops_2B' in column, df.columns)

if len(list(matching)) == 1:
    df.drop("Ecology_2B", axis=1, inplace=True)
    df.drop("Ecology_3B", axis=1, inplace=True)
    df.drop("Shops_2B", axis=1, inplace=True)

df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,Ecology_2A,Ecology_3A,Shops_2A
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,33,7976,5,900.0,0,11,184966.930730,0,0,0
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,46,10309,1,240.0,1,16,300009.450063,0,0,0
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,34,7759,0,229.0,1,3,220925.908524,0,0,0
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,23,5735,3,1084.0,0,5,175616.227217,0,0,0
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,35,5776,1,2078.0,2,4,150226.531644,0,0,0


Подготовка 

In [143]:
X = df.copy().drop(['Price'], axis=1)
y = df[['Price']]

In [144]:
from sklearn.model_selection import train_test_split

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [146]:
from sklearn.ensemble import RandomForestRegressor

In [147]:
rfr = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

In [148]:
rfr.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)